In [14]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import torch 
import torch.jit
import torch.nn as nn 
import torchvision
from torchvision import transforms, datasets
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import torch 
import torch.jit
import torch.nn as nn 
import torchvision
from torchvision import transforms, datasets
import sklearn.metrics as metrics
from sklearn.svm import SVC
from sklearn import svm
from sklearn import datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
# from sklearn.linear_model import 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.impute import KNNImputer
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import ClusterCentroids
import math
import xgboost as xgb
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

print(tf.__version__)

from tensorflow.python.client import device_lib
print("tensorflow doesn't detects GPU: ")
print(device_lib.list_local_devices() )
print(tf.test.gpu_device_name())
print(tf.config.list_physical_devices())

print("torch detects GPU: " , torch.cuda.is_available())



2.3.0
tensorflow doesn't detects GPU: 
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7539944681906525294
]

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
torch detects GPU:  True


In [15]:
import pandas as pd
import numpy as np
df = pd.read_csv("train_features_imputed_normalized.csv")
labels_df = pd.read_csv("train_labels.csv")
X = np.array(df.iloc[:, 1:].values,  dtype=float)
X_train = X[:int(X.shape[0]*0.8)]
X_valid = X[int(X.shape[0]*0.8):]
Y = np.array(labels_df.iloc[:, 1:].values,  dtype=float)
Y_train = Y[:int(Y.shape[0]*0.8)]
Y_valid = Y[int(Y.shape[0]*0.8):]



In [16]:
col = 10

In [17]:
clf_xgb = xgb.XGBRFRegressor(max_depth=10)
# clf_xgb = xgb.XGBClassifier(objective='binary:logistic', max_depth=3, n_estimators=300, learning_rate=0.05)
clf_xgb.fit(X_train, Y_train[:,10],verbose=True, eval_metric="auc", eval_set=[(X_valid, Y_valid[:,col])])

[0]	validation_0-auc:0.68636


XGBRFRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bytree=1, enable_categorical=False, gamma=0, gpu_id=-1,
               importance_type=None, interaction_constraints='',
               max_delta_step=0, max_depth=10, min_child_weight=1, missing=nan,
               monotone_constraints='()', n_estimators=100, n_jobs=6,
               num_parallel_tree=100, objective='reg:squarederror',
               predictor='auto', random_state=0, reg_alpha=0,
               scale_pos_weight=1, tree_method='exact', validate_parameters=1,
               verbosity=None)

In [18]:
pred = clf_xgb.predict(X_valid)
print(pred)
score = metrics.roc_auc_score(Y_valid[:,col], pred)
score

[0.04021233 0.0273331  0.10143197 ... 0.03193904 0.03128313 0.02972065]


0.686355791542457

In [19]:

# specify parameters via map
param = {'booster': 'dart',
         'max_depth': 5, 'learning_rate': 0.1,
         'objective': 'binary:logistic', 'silent': True,
         'sample_type': 'uniform',
         'normalize_type': 'tree',
         'rate_drop': 0.1,
         'skip_drop': 0.5}
num_round = 50
bst = xgb.train(param, xgb.DMatrix(X_train, label=Y_train[:,col]), num_round)
# make prediction
# ntree_limit must not be 0
preds = bst.predict(xgb.DMatrix(X_valid, label=Y_valid[:,col]))

[13:25:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[13:25:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [20]:
print(preds)
score = metrics.roc_auc_score(Y_valid[:,col], preds)
score

[0.03662579 0.040127   0.16152854 ... 0.04153312 0.04501774 0.03768276]


0.689327189599694